Step 1: Install Dependencies

In [4]:
!pip install transformers datasets sentencepiece fastapi uvicorn nest-asyncio mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 18.3 MB/s eta 0:00:00


Step 2: Import Libraries

In [5]:
import json
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

 Step 3: Prepare Dataset

In [6]:
# Sample dataset: Add more rows as needed
data = [
    {"input_text": "Show me a 2BHK flat in Andheri under 1 crore",
     "target_text": "SELECT * FROM properties WHERE type='2BHK' AND location='Andheri' AND price<10000000"},
    {"input_text": "Any 1BHK apartment listings in Powai below 75 lakh?",
     "target_text": "SELECT * FROM properties WHERE type='1BHK' AND location='Powai' AND price<7500000"},
    {
        "input_text": "Show me a 2BHK flat in Andheri under 1 crore",
        "target_text": "SELECT * FROM properties WHERE type='2BHK' AND location='Andheri' AND price<10000000"
    },
    {
        "input_text": "Any 1BHK apartment listings in Powai below 75 lakh?",
        "target_text": "SELECT * FROM properties WHERE type='1BHK' AND location='Powai' AND price<7500000"
    },
    {
    "input_text": "Need a 2BHK in Goregaon below 77 lakhs",
    "target_text": "SELECT * FROM properties WHERE type='2BHK' AND location='Goregaon' AND price<7700000"
    },
    {
      "input_text": "Do you have any Studio apartments in Kurla under ₹5900000?",
      "target_text": "SELECT * FROM properties WHERE type='Studio' AND location='Kurla' AND price<5900000"
    },
    {
      "input_text": "Find me a 1BHK flat in Vile Parle under 86 lakhs",
      "target_text": "SELECT * FROM properties WHERE type='1BHK' AND location='Vile Parle' AND price<8600000"
    },
    {
      "input_text": "Looking for 3BHK properties in Dahisar priced under ₹7600000",
      "target_text": "SELECT * FROM properties WHERE type='3BHK' AND location='Dahisar' AND price<7600000"
    },
    {
      "input_text": "Suggest Studio options near Powai under ₹6100000",
      "target_text": "SELECT * FROM properties WHERE type='Studio' AND location='Powai' AND price<6100000"
    },
    {
      "input_text": "List 2BHK homes in Vile Parle below ₹9200000",
      "target_text": "SELECT * FROM properties WHERE type='2BHK' AND location='Vile Parle' AND price<9200000"
    },
    {
      "input_text": "Searching for 3BHK flats in Kurla under 88 lakhs",
      "target_text": "SELECT * FROM properties WHERE type='3BHK' AND location='Kurla' AND price<8800000"
    },
    {
      "input_text": "Show me 1BHK apartments in Andheri for less than ₹4100000",
      "target_text": "SELECT * FROM properties WHERE type='1BHK' AND location='Andheri' AND price<4100000"
    },
    {
      "input_text": "Any Studio flat in Vile Parle below ₹3600000?",
      "target_text": "SELECT * FROM properties WHERE type='Studio' AND location='Vile Parle' AND price<3600000"
    },
    {
      "input_text": "Looking for a 2BHK in Dahisar under 62 lakhs",
      "target_text": "SELECT * FROM properties WHERE type='2BHK' AND location='Dahisar' AND price<6200000"
    }
]

# Convert to HuggingFace Dataset
ds = Dataset.from_list(data)


Step 4: Load Tokenizer & Model (Pretrained T5)

In [7]:
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load model + tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


 Step 5: Preprocess Data

In [8]:
# Load data
ds = Dataset.from_list(data)

# Tokenization
def preprocess(example):
    inputs = tokenizer("translate to SQL: " + example["input_text"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(example["target_text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

ds = ds.map(preprocess)

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Step 6: Define Training Arguments

In [9]:
args = TrainingArguments(
    output_dir="./nl2sql-t5",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    save_total_limit=1,
    save_strategy="epoch",
    logging_steps=5,
    overwrite_output_dir=True,
    remove_unused_columns=False
)

 Step 7: Train the Model

In [ ]:
trainer = Trainer(model=model, args=args, train_dataset=ds)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bookpoole (bookpoole-gtt-data-solution) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
5,11.132500
10,10.533800
15,8.240300
20,6.865000
25,5.837500
30,5.638000


Step 8: Save Model and Tokenizer

In [10]:
model.save_pretrained("/content/nl2sql-t5")
tokenizer.save_pretrained("/content/nl2sql-t5")
print("✅ Model saved to /content/nl2sql-t5")


✅ Model saved to /content/nl2sql-t5


 Step 9: Run Inference on Sample Input

In [11]:
model = T5ForConditionalGeneration.from_pretrained("/content/nl2sql-t5")
tokenizer = T5Tokenizer.from_pretrained("/content/nl2sql-t5")

def query_to_sql(nl_query):
    input_ids = tokenizer("translate to SQL: " + nl_query, return_tensors="pt").input_ids
    output = model.generate(input_ids, max_length=64)
    return tokenizer.decode(output[0], skip_special_tokens=True)

print("SQL:", query_to_sql("I want a Studio flat in Bandra below 50 lakh"))

SQL: SQL: Ich möchte eine Studio-Apartment in Bandra unter 50 lakh.


 Step 10: Deploy FastAPI (Run in Colab Cell)

In [12]:
from fastapi import FastAPI, Request
import uvicorn
import nest_asyncio
from threading import Thread

app = FastAPI()

@app.post("/query")
async def process_query(req: Request):
    body = await req.json()
    nl = body["query"]
    sql = query_to_sql(nl)
    return {"sql_query": sql}

# Run API server
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

nest_asyncio.apply()
Thread(target=run).start()


Step 11: Test the API

In [13]:
import requests

response = requests.post("http://127.0.0.1:8000/query", json={"query": "Show me 2BHK flat in Goregaon under 50 lakh"})
print(response.json())


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-6' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-12-3799947767.py", line 17, in 

INFO:     127.0.0.1:41140 - "POST /query HTTP/1.1" 200 OK
{'sql_query': 'SQL SQL: Zeigen Sie mir 2BHK Wohnung in Goregaon unter 50 lakh'}
